In [1]:
import numpy as np
from sklearn import datasets

In [2]:
boston = datasets.load_boston()
X = boston.data
y = boston.target

X = X[y<50.0]
y = y[y<50.0]

In [3]:
import sys
sys.path.append('..')
from model_selection_tools.split_data_enc import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, seed=666)

In [4]:
from model_selection_tools.linear_regression import LinearRegression

lin_reg1 = LinearRegression()
%time lin_reg1.fit_normal(X_train, y_train)
lin_reg1.score(X_test, y_test)

Wall time: 20 ms


0.8129802602658509

### 梯度下降法

In [5]:
lin_reg2 = LinearRegression()
lin_reg2.fit_gd(X_train, y_train)

D:\Anaconda\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
..\model_selection_tools\linear_regression.py:42: RuntimeWarning: overflow encountered in square
  return np.sum((y - X_b.dot(theta)) ** 2) / len(y)
..\model_selection_tools\linear_regression.py:62: RuntimeWarning: invalid value encountered in double_scalars
  if(abs(J(theta, X_b, y) - J(last_theta, X_b, y))< epsilon):


LinearRegression()

In [6]:
lin_reg2.coef_  # 发现系数都是Nan， 可想到是因为数太大发生了溢出

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [7]:
X_train[:3,:] # 特征对应数据规模不一样，有一些是零点几的维度，有的是几百
# 因此实际求得的梯度还是太大，用默认的eta还是步长太大，使得我们的梯度下降是不收敛的

array([[1.42362e+01, 0.00000e+00, 1.81000e+01, 0.00000e+00, 6.93000e-01,
        6.34300e+00, 1.00000e+02, 1.57410e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.96900e+02, 2.03200e+01],
       [3.67822e+00, 0.00000e+00, 1.81000e+01, 0.00000e+00, 7.70000e-01,
        5.36200e+00, 9.62000e+01, 2.10360e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.80790e+02, 1.01900e+01],
       [1.04690e-01, 4.00000e+01, 6.41000e+00, 1.00000e+00, 4.47000e-01,
        7.26700e+00, 4.90000e+01, 4.78720e+00, 4.00000e+00, 2.54000e+02,
        1.76000e+01, 3.89250e+02, 6.05000e+00]])

In [8]:
lin_reg2.fit_gd(X_train, y_train, eta=0.000001)

LinearRegression()

In [9]:
lin_reg2.coef_

array([-0.10478361,  0.1152641 , -0.06249712,  0.00207336,  0.00446563,
        0.11944014,  0.04677988,  0.03450425, -0.00428707,  0.00327022,
        0.12699404,  0.04485901, -0.22537907])

In [10]:
lin_reg2.score(X_test, y_test) # 显然此时找到的西塔还没达到损失函数的最小值
#是因为我们现在的eta又太小了，导致每一步行进太小

0.27556634853389206

In [11]:
%time lin_reg2.fit_gd(X_train, y_train, eta=0.000001, n_iters=1e6)

Wall time: 1min 22s


LinearRegression()

In [12]:
lin_reg2.score(X_test, y_test) # 可发现仍没达到最优， 我们需要更多的循环，但又太耗时了

0.7541852353980765

#### 真正的问题是我们的数据整体不在一个规模上，因此在使用梯度下降法前，最好进行数据归一化

In [13]:
from sklearn.preprocessing import StandardScaler

In [15]:
standardScaler = StandardScaler()
standardScaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [16]:
X_train_standard = standardScaler.transform(X_train)

In [17]:
lin_reg3 = LinearRegression()
%time lin_reg3.fit_gd(X_train_standard, y_train)

Wall time: 563 ms


LinearRegression()

In [18]:
X_test_standard = standardScaler.transform(X_test)

In [19]:
lin_reg3.score(X_test_standard, y_test) # 与使用正规方程得到的结果一致，且非常快速

0.8129880620122235

### 梯度下降法的优势

In [20]:
m = 1000 # 1000个样本
n = 5000 # 每个样本5000个特征

big_X = np.random.normal(size=(m,n)) #自动生成的数据其实已经归一化了

true_theta = np.random.uniform(0.0, 100.0, size = n+1)
big_y = big_X.dot(true_theta[1:]) + true_theta[0] + np.random.normal(0., 10., size=m) 
# 再加上一个均值为0 标准差为10 服从正态分布的噪音向量

In [21]:
big_reg1 = LinearRegression()
%time big_reg1.fit_normal(big_X, big_y)

Wall time: 23.4 s


LinearRegression()

In [22]:
big_reg2 = LinearRegression()
%time big_reg2.fit_gd(big_X, big_y)

Wall time: 8.38 s


LinearRegression()

#### 此时可发现用梯度下降法来训练，比使用正规方程是要小的

如果加大特征数，梯度下降法的优势对比正规方程法会更加明显

现在：样本数量，小于特征数量 -> 因为现在使用的梯度下降法使用的公式在计算<br>
梯度时要让每一个样本都参与计算。这使得当样本数量较大时，计算梯度也会很慢<br>
<br>
改进方法：随机梯度下降法